# Regression Week 5: Ass1: Feature Selection and LASSO (Interpretation)
###sklearn version using pandas  - using scikit-learn with Pandas see http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

PWD : TNKSBZ9CT Receipt number 160899085221, 339411363031

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties. The algorithm used to fit the model is Coordinate Descent.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create
**0.** Load the sales dataset:

In [1]:
import pandas as pd
import numpy as np
import math
import random
from sklearn import linear_model

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 
              'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('../Week3/kc_house_data.csv', dtype=dtype_dict)
print "\nNumber of features = %d" %len(sales.columns)
sales.head(2)


Number of features = 21


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0


# Create new features

**1.** Create 4 new features by performing following transformation on inputs: As in Week 2, we consider features that are some transformations of inputs. 

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']
print sales.columns, "\nNumber of features = %d" %len(sales.columns)

Index([u'id', u'date', u'price', u'bedrooms', u'bathrooms', u'sqft_living',
       u'sqft_lot', u'floors', u'waterfront', u'view', u'condition', u'grade',
       u'sqft_above', u'sqft_basement', u'yr_built', u'yr_renovated',
       u'zipcode', u'lat', u'long', u'sqft_living15', u'sqft_lot15',
       u'sqft_living_sqrt', u'sqft_lot_sqrt', u'bedrooms_square',
       u'floors_square'],
      dtype='object') 
Number of features = 25


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

**2.** Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.
Let us fit a model with all the features available, plus the features we just created above.

In [4]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']
print "We are only going to use %d features for training and testing." %len(all_features)

We are only going to use 17 features for training and testing.


Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters and initialize 
model_all.fit(sales[all_features], sales['price']) # learn weights by fitting to all data

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [6]:
zip(all_features, model_all.coef_)
#print  model_all.coefficients[model_all.coefficients['value'] != 0]

[('bedrooms', 0.0),
 ('bedrooms_square', 0.0),
 ('bathrooms', 0.0),
 ('sqft_living', 134.43931395540994),
 ('sqft_living_sqrt', 0.0),
 ('sqft_lot', 0.0),
 ('sqft_lot_sqrt', 0.0),
 ('floors', 0.0),
 ('floors_square', 0.0),
 ('waterfront', 0.0),
 ('view', 24750.004585613766),
 ('condition', 0.0),
 ('grade', 61749.103090711593),
 ('sqft_above', 0.0),
 ('sqft_basement', 0.0),
 ('yr_built', -0.0),
 ('yr_renovated', 0.0)]

In [7]:
print model_all.sparse_coef_
#len(model_all.sparse_coef_)
print "(model_all.coef_).shape[0]", (model_all.coef_).shape[0]
#model_all.sparse_coef_.getnnz
print "model_all.sparse_coef_.shape:", model_all.sparse_coef_.shape

  (0, 3)	134.439313955
  (0, 10)	24750.0045856
  (0, 12)	61749.1030907
(model_all.coef_).shape[0] 17
model_all.sparse_coef_.shape: (1, 17)


Note that a majority of the weights have been set to zero. Only 3 out of initial 17 are non-zero. So by **setting a very large L1 penalty**, we are performing a **subset or feature selection**. 

**3. QUIZ QUESTION Q1: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?**
According to this list of weights, which of the features have been chosen?  'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above'

 
Which of the following features have been chosen by LASSO, i.e. which features were assigned nonzero weights? (Choose all that apply)
* yr_renovated
* waterfront
* sqft_living   <----
* grade         <----
* floors

# Selecting an L1 penalty

**4.** To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
testing = pd.read_csv('../Week3/wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('../Week3/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('../Week3/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [9]:
# Make sure to create the 4 features in ALL 3 data sets:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

**5.** Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty=5e2` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [22]:
penalty_rss = []
min_rss = None
best_l1_penalty = None
best_l1_model = None

for l1_penalty in np.logspace(1, 7, num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True) # set parameters and initialize model
    model.fit(training[all_features], training['price'])  # learn weights by fitting to training data
    predictions =  model.predict(validation[all_features])# prediction on validation set
    residuals = validation['price'] - predictions
    rss = (residuals**2).sum()
    #RSS.append(rss)
    num_non_zeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    
    penalty_rss.append((l1_penalty, rss))
    print "\nl1_penalty = %0.2f: non-zero coefficients = %d: rss= %0.2f" \
          %(l1_penalty, num_non_zeros, rss)    
    
    if min_rss is None or rss < min_rss:
        min_rss = rss
        best_l1_penalty = l1_penalty
        best_l1_model = model
      
print "\n\nOverall Best l1_penalty on validation data = %0.2f, as it gives lowest rss of %0.2f" %(best_l1_penalty,min_rss)    


l1_penalty = 10.00: non-zero coefficients = 15: rss= 398213327300134.19

l1_penalty = 31.62: non-zero coefficients = 15: rss= 399041900253348.19

l1_penalty = 100.00: non-zero coefficients = 11: rss= 429791604072558.12

l1_penalty = 316.23: non-zero coefficients = 6: rss= 463739831045119.62

l1_penalty = 1000.00: non-zero coefficients = 4: rss= 645898733633810.38

l1_penalty = 3162.28: non-zero coefficients = 1: rss= 1222506859427156.75

l1_penalty = 10000.00: non-zero coefficients = 1: rss= 1222506859427156.75

l1_penalty = 31622.78: non-zero coefficients = 1: rss= 1222506859427156.75

l1_penalty = 100000.00: non-zero coefficients = 1: rss= 1222506859427156.75

l1_penalty = 316227.77: non-zero coefficients = 1: rss= 1222506859427156.75

l1_penalty = 1000000.00: non-zero coefficients = 1: rss= 1222506859427156.75

l1_penalty = 3162277.66: non-zero coefficients = 1: rss= 1222506859427156.75

l1_penalty = 10000000.00: non-zero coefficients = 1: rss= 1222506859427156.75


Overall Best l1

In [24]:
penalty_rss = sorted(penalty_rss, key = lambda x : x[1])
non_zeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

l1_penalty_best = penalty_rss[0][0]
print(l1_penalty_best)

print "The best model has", non_zeros, "non zero coefficients, its RSS on validation is",\
min_rss, "for a lambda of", best_l1_penalty

 10.0
The best model has 1 non zero coefficients, its RSS on validation is 3.982133273e+14 for a lambda of 10.0


*** 6. QUIZ QUESTIONS Q2: Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on VALIDATION data?***
1. What was the best value for the `l1_penalty`? 10.0
2. What is the RSS on TEST data of the model with the best `l1_penalty`? 514597632122622.88 (sklearn) 625766285142460.50(gl)


In which of the following ranges does the best l1_penalty fall?
* Between 0 and 100    <----
* Between 100 and 1000
* Between 1000 and 10000
* Between 10000 and 100000
* Greater than 100000

**7.** Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty - dome see above.  We had saved the model for best L1 earlier so we can just predict using that model on test data.

In [11]:
predictions =  best_l1_model.predict(testing[all_features])  # prediction on validation
residuals = testing['price'] - predictions
rss = (residuals**2).sum()
print rss       #1.56983602382e+14   much higher test RSS using graphlabs

9.84674025527e+13


Or we can use best L1 identified earlier and train on training data using that L1, then test on test data. Shud get same results.

In [12]:
model_best_l1 = linear_model.Lasso(alpha=best_l1_penalty, normalize=True) # set parameters and initialize model
model_best_l1.fit(training[all_features], training['price'])  # learn weights using best_l1_penalty

predictions =  model_best_l1.predict(testing[all_features])  # prediction on test data
residuals = testing['price'] - predictions
rss = (residuals**2).sum()
print rss       #1.56983602382e+14   much higher test RSS using graphlabs

9.84674025527e+13


**8. QUIZ QUESTION: Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero.** 

 A succinct way to do this using numpy is 
 
 *np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)*
 
 15 non-zero, have 17 features so 17 coef plus 1 intercept - tolal 18, 15 non-zero

In [13]:
#print len(model_best_l1.coefficients[model_best_l1.coefficients['value'] != 0])
#print model_best_l1.coefficients[model_best_l1.coefficients['value'] != 0]. \
#                                      print_rows(num_rows=20, num_columns=4)
print model_best_l1.coef_
print "Intercept", model_best_l1.intercept_
print model_best_l1.sparse_coef_

# NOte unlike with graph labs implementation of Lasso - with sklearn 3 coeffificients are exactly zero!!!
non_zeros = np.count_nonzero(model_best_l1.coef_) + np.count_nonzero(model_best_l1.intercept_)
print non_zeros

[ -1.61445628e+04   3.73245384e+02   5.08412433e+04   6.17853560e+02
  -4.44113549e+04   7.85623065e-01  -7.01194765e+02  -0.00000000e+00
   5.01420046e+03   6.19488752e+05   3.80418557e+04   2.49987718e+04
   1.28716235e+05   0.00000000e+00   0.00000000e+00  -3.29383118e+03
   1.00573209e+01]
Intercept 6630155.66863
  (0, 0)	-16144.5627571
  (0, 1)	373.245384349
  (0, 2)	50841.2433399
  (0, 3)	617.853559504
  (0, 4)	-44411.3548667
  (0, 5)	0.785623064832
  (0, 6)	-701.194765368
  (0, 8)	5014.20045697
  (0, 9)	619488.752486
  (0, 10)	38041.8556525
  (0, 11)	24998.7718382
  (0, 12)	128716.234621
  (0, 15)	-3293.83117995
  (0, 16)	10.0573208643
15


# Limit the number of nonzero weights

**9.** What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

**10.** Assign 7 to the variable ‘max_nonzeros’.

In [14]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

**11.** Exploring large range of l1_penalty, so let's define a wide range of possible `l1_penalty_values`:

In [15]:
l1_penalty_values = np.logspace(1, 4, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [25]:
nonzero_list = list()
l1_penalty_min = None
l1_penalty_max = None

for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True) # set parameters and initialize model
    model.fit(training[all_features], training['price'])  # learn weights

    num_nonzero_coef = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    nonzero_list.append( (l1_penalty, num_nonzero_coef) )
    # create a list of tuples

    print "\nl1_penalty = %0.2f: non-zero coefficients = %d" \
          %(l1_penalty, num_nonzero_coef)    
           
    if ( l1_penalty_min  is None) or (num_nonzero_coef > max_nonzeros):
        l1_penalty_min = l1_penalty
        nonzero_coef_max = num_nonzero_coef
    
    if (l1_penalty_max is None) and (num_nonzero_coef < max_nonzeros):
        l1_penalty_max = l1_penalty
        nonzero_coef_min = num_nonzero_coef
        
print "\n", nonzero_list       
print "\nThe largest l1_penalty that has more non-zeros than %d is %0.2f with %d non-zero coefs" \
%(max_nonzeros, l1_penalty_min, nonzero_coef_max )    
print "\nThe smallest l1_penalty that has fewer non-zeros than %d is %0.2f with %d non-zero coefs" \
%(max_nonzeros, l1_penalty_max, nonzero_coef_min)  


l1_penalty = 127.43: non-zero coefficients = 10

l1_penalty = 134.60: non-zero coefficients = 10

l1_penalty = 141.77: non-zero coefficients = 8

l1_penalty = 148.94: non-zero coefficients = 8

l1_penalty = 156.11: non-zero coefficients = 7

l1_penalty = 163.28: non-zero coefficients = 7

l1_penalty = 170.45: non-zero coefficients = 7

l1_penalty = 177.62: non-zero coefficients = 7

l1_penalty = 184.79: non-zero coefficients = 7

l1_penalty = 191.96: non-zero coefficients = 7

l1_penalty = 199.13: non-zero coefficients = 7

l1_penalty = 206.30: non-zero coefficients = 6

l1_penalty = 213.47: non-zero coefficients = 6

l1_penalty = 220.64: non-zero coefficients = 6

l1_penalty = 227.81: non-zero coefficients = 6

l1_penalty = 234.98: non-zero coefficients = 6

l1_penalty = 242.15: non-zero coefficients = 6

l1_penalty = 249.32: non-zero coefficients = 6

l1_penalty = 256.49: non-zero coefficients = 6

l1_penalty = 263.67: non-zero coefficients = 6

[(127.42749857031335, 10), (134.59789

**12.** Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)

*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above.... we did this above in the loop
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries - we do this below..

In [26]:
# just showing off list comprehension that also works and for loop will be much smaller above
# this uses list of tuple created in loop above
# `nonzero_list.append( (l1_penalty, num_nonzero_coef) )` 
l1_penalty_min = max([t[0] for t in nonzero_list if t[1] > max_nonzeros])
l1_penalty_max = min([t[0] for t in nonzero_list if t[1] < max_nonzeros])
print(l1_penalty_min, l1_penalty_max)

(148.93869719705901, 206.30189353504741)


In [27]:
print l1_penalty_min
print l1_penalty_max
#2976351441.63 using gl we got these
#3792690190.73

148.938697197
206.301893535


***13. QUIZ QUESTIONS Q4
What values did you find for `l1_penalty_min` and `l1_penalty_max`? ***

If you are using GraphLab Create, enter your answer in simple decimals without commas (e.g. 1131000000), rounded to nearest millions.

If you are using scikit-learn, enter your answer in simple decimals without commas (e.g. 4313), rounded to nearest integer. 
`l1_penalty_min` 149 ,`l1_penalty_max` 206

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

**14.** Exploring narrower range/region of `l1_penalty` values we found:between ‘l1_penalty_min’ and ‘l1_penalty_max’. We look for the L1 penalty in this range that produces **exactly the right number of nonzeros AND also minimizes RSS on the VALIDATION set**.

In [18]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

**15.** Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [42]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
penalty_rss_model = []
min_rss = None
best_l1_penalty = None
best_model = None

for l1_penalty in l1_penalty_values :
    
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True) # set parameters and initialize model
    model.fit(training[all_features], training['price'])         # learn weights

    num_nonzero_coef = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    # nonzero_list.append(num_nonzero_coef)
       
    predictions = model.predict(validation[all_features])
    residuals = validation['price'] - predictions
    rss = (residuals**2).sum()
    
    print "\nl1_penalty = %0.2f: non-zero coefficients = %d: rss= %0.2f" \
          %(l1_penalty, num_nonzero_coef, rss)
        
    if (num_nonzero_coef == max_nonzeros) and (min_rss is None or rss < min_rss):
        min_rss = rss
        best_l1_penalty = l1_penalty
        best_model = model
        penalty_rss_model.append((l1_penalty, rss, model))

        
print "\n\nOverall Best l1_penalty on validation data = %0.2f, as it gives lowest rss of %0.2f" %(best_l1_penalty,min_rss)    


l1_penalty = 148.94: non-zero coefficients = 8: rss= 439158937799659.38

l1_penalty = 151.96: non-zero coefficients = 8: rss= 439566717107617.06

l1_penalty = 154.98: non-zero coefficients = 7: rss= 439925583107807.88

l1_penalty = 158.00: non-zero coefficients = 7: rss= 440228446594392.94

l1_penalty = 161.02: non-zero coefficients = 7: rss= 440538757277617.25

l1_penalty = 164.03: non-zero coefficients = 7: rss= 440858188572511.06

l1_penalty = 167.05: non-zero coefficients = 7: rss= 441186575552993.31

l1_penalty = 170.07: non-zero coefficients = 7: rss= 441523902274328.62

l1_penalty = 173.09: non-zero coefficients = 7: rss= 441870188649273.94

l1_penalty = 176.11: non-zero coefficients = 7: rss= 442225434677828.06

l1_penalty = 179.13: non-zero coefficients = 7: rss= 442589625511468.50

l1_penalty = 182.15: non-zero coefficients = 7: rss= 442962820579928.69

l1_penalty = 185.17: non-zero coefficients = 7: rss= 443344976391377.56

l1_penalty = 188.19: non-zero coefficients = 7: rs

In [43]:
non_zeros = np.count_nonzero(best_model.coef_) + np.count_nonzero(best_model.intercept_)

print "The best model has", non_zeros, "non zero coefficients, its RSS is on validation", \
min_rss, "for a lambda of", best_l1_penalty

print best_model.coef_
print "Intercept", model_best_l1.intercept_
print best_model.sparse_coef_

The best model has 7 non zero coefficients, its RSS is on validation 4.39925583108e+14 for a lambda of 154.976928391
[ -0.00000000e+00  -0.00000000e+00   1.08488432e+04   1.63305308e+02
   0.00000000e+00  -0.00000000e+00  -0.00000000e+00   0.00000000e+00
   0.00000000e+00   5.07353781e+05   4.19793395e+04   0.00000000e+00
   1.16371869e+05   0.00000000e+00   0.00000000e+00  -2.62047793e+03
   0.00000000e+00]
Intercept 6630155.66863
  (0, 2)	10848.8431638
  (0, 3)	163.305307806
  (0, 9)	507353.780812
  (0, 10)	41979.339539
  (0, 12)	116371.869245
  (0, 15)	-2620.47792827


In [44]:
zip(all_features, best_model.coef_)   #only 6 listed here cause 7th in the intercept

[('bedrooms', -0.0),
 ('bedrooms_square', -0.0),
 ('bathrooms', 10848.843163831263),
 ('sqft_living', 163.30530780619353),
 ('sqft_living_sqrt', 0.0),
 ('sqft_lot', -0.0),
 ('sqft_lot_sqrt', -0.0),
 ('floors', 0.0),
 ('floors_square', 0.0),
 ('waterfront', 507353.78081239725),
 ('view', 41979.339539015469),
 ('condition', 0.0),
 ('grade', 116371.86924520371),
 ('sqft_above', 0.0),
 ('sqft_basement', 0.0),
 ('yr_built', -2620.4779282734412),
 ('yr_renovated', 0.0)]

***16. QUIZ QUESTIONS 5 and 6***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?

2. What features in this model have non-zero coefficients?

If you are using GraphLab Create, enter your answer in simple decimals without commas (e.g. 1131000000), rounded to nearest millions.  3448000000

If you are using scikit-learn, enter your answer in simple decimals without commas (e.g. 4342), rounded to nearest integer. 156

Which of the following features has non-zero coefficients? (Choose all that apply)
* sqft_living
* bedrooms_square
* sqft_lot_sqrt  
* bathrooms      <<----
* floors

In [51]:
# we had built a list of tuples in loop above
# penalty_rss_model.append((l1_penalty, rss, model))
# We can interrogate that list to get lowest rss and its associated lambda and model
penalty_rss_model = sorted(penalty_rss_model, key = lambda x : x[1])
lowest_rss = penalty_rss_model[0][1]
l1_penalty_optimum = penalty_rss_model[0][0]
model_optimum = penalty_rss_model[0][2]
print "Lowest rss for 7 non-zero coefs = %d, with penalty %d and number of non-zero coeficcients %d" \
%(lowest_rss, l1_penalty_optimum, (np.count_nonzero(model_optimum.coef_) + np.count_nonzero(model_optimum.intercept_)) )

Lowest rss for 7 non-zero coefs = 439925583107807, with penalty 154 and number of non-zero coeficcients 7


In [21]:
zip(all_features,best_model.coef_)

[('bedrooms', -0.0),
 ('bedrooms_square', -0.0),
 ('bathrooms', 10610.890284398485),
 ('sqft_living', 163.38025164762911),
 ('sqft_living_sqrt', 0.0),
 ('sqft_lot', -0.0),
 ('sqft_lot_sqrt', -0.0),
 ('floors', 0.0),
 ('floors_square', 0.0),
 ('waterfront', 506451.68711485656),
 ('view', 41960.043554851516),
 ('condition', 0.0),
 ('grade', 116253.55369970751),
 ('sqft_above', 0.0),
 ('sqft_basement', 0.0),
 ('yr_built', -2612.2348803574919),
 ('yr_renovated', 0.0)]